In [32]:
%load_ext autoreload
%autoreload 1
%aimport game

import game
import time
import random

game._build_row_tables()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
def noskill(bs, action_space):
    return random.choice(action_space)(bs)

def naive_minimize_empty_tiles(bs, action_space):
    best_action, best_score = None, 0
    for action in action_space:
        new_bs = action(bs)
        num_empty_tiles = len(game.get_empty_tiles(new_bs))
        if num_empty_tiles > best_score:
            best_action, best_score = action, num_empty_tiles
    return best_action(bs)


In [34]:
class Game:
    def __init__(self, board = None):
        if board is None:
            self.start_board = [
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
            ]
        
        self.start_bitset = game.to_bitset(self.start_board)

    def watch_game(self):
        bs = self.start_bitset

        for i in range(100000):
            bs = game.generate_tile(bs)
            action_space = game.get_action_space(bs)
            game.print_board(game.to_board(bs), f"Turn {i}: Random tile")
            if action_space:
                bs = noskill(bs, action_space)
            else:
                break
            game.print_board(game.to_board(bs), f"Turn {i}: Taken action")
            
    def run_game(self, ai, max_iters=1000000, num_games=10000):
        game_results = []

        for game_index in range(num_games):
            bs = self.start_bitset

            start_iter = time.time()

            for i in range(max_iters):
                bs = game.generate_tile(bs)
                action_space = game.get_action_space(bs)
                if action_space:
                    bs = ai(bs, action_space)
                else:
                    break
                
            time_taken = time.time() - start_iter
            game_results.append({
                'num_turns_taken': i,
                'max_tile_reached': game.get_max_tile(bs),
                'time_taken': time_taken,
            })

        return game_results, ai.__name__
        
    def print_results(self, game_results, ai_name):
        print(f'Test: {ai_name}')

        num_games = len(game_results)
        print()
        print(f'Number of games played: {num_games}')
        print(f'Total time taken: {sum(result["time_taken"] for result in game_results)} seconds')
        print(f'Games per second: {num_games / sum(result["time_taken"] for result in game_results)}')
        print(f'Turns per second: {sum(result["num_turns_taken"] for result in game_results) / sum(result["time_taken"] for result in game_results)}')

        print()
        print(f'Max number of turns before game over: {max(result["num_turns_taken"] for result in game_results)}')
        print(f'Min number of turns before game over: {min(result["num_turns_taken"] for result in game_results)}')
        print(f'Average number of turns before game over: {sum(result["num_turns_taken"] for result in game_results) / len(game_results)}')

        print()
        print(f'Max tile reached: {max(result["max_tile_reached"] for result in game_results)}')
        print(f'Min tile reached: {min(result["max_tile_reached"] for result in game_results)}')
        print(f'Average max tile reached: {sum(result["max_tile_reached"] for result in game_results) / len(game_results)}')

    def plot_results(self, game_results):
        import pandas as pd
        import matplotlib.pyplot as plt

        df = pd.DataFrame(game_results)

        plt.hist(df['max_tile_reached'], bins=100)
        plt.title('Max tile reached')
        plt.show()

        plt.hist(df['num_turns_taken'], bins=100)
        plt.title('Number of turns before game over')
        plt.show()

In [ ]:
g = Game()
results, ai_name = g.run_game(noskill)
g.print_results(results, ai_name)
g.plot_results(results)